# Additional Data Creator Workbook

In [1]:
# Import the relevent packages
import os # File/directory operations
import ntpath # File/directory operations
import imgaug as ia # Image augmentations
import imgaug.augmenters as iaa # Image augmentations
import imageio as imio # Reading images
import numpy as np # Matrix operations
from skimage import data # Image operations
from skimage.color import rgb2gray # RGB image to grey
from skimage.transform import resize # Resize images
import matplotlib.pyplot as plt # for visualisation
import pandas as pd # Data manipulations
from scipy import stats #not used but helps with debuging
from tqdm import tqdm # Progress bar
import warnings # Hide warnings in jupyter
warnings.filterwarnings('ignore')

In [2]:
# Set the input and output paths for later reference
dataPaths = {'InputImageTrain':os.path.abspath(os.path.join('Data','Train','Images',)),
             'InputAnnotTrain':os.path.abspath(os.path.join('Data','Train','Annotations')),
             'InputImageTest':os.path.abspath(os.path.join('Data','Test','Images')),
             'InputAnnotTest':os.path.abspath(os.path.join('Data','Test','Annotations')),
             'OutputImageTrain':os.path.abspath(os.path.join('Output','Train','Images')),
             'OutputAnnotTrain':os.path.abspath(os.path.join('Output','Train','Annotations')),
             'OutputImageTest':os.path.abspath(os.path.join('Output','Test','Images')),
             'OutputAnnotTest':os.path.abspath(os.path.join('Output','Test','Annotations')),
             'XTrain':os.path.abspath(os.path.join('OutputData','XTrain')),
             'yTrain':os.path.abspath(os.path.join('OutputData','yTrain')),
             'XTest':os.path.abspath(os.path.join('OutputData','XTest')),
             'yTest':os.path.abspath(os.path.join('OutputData','yTest'))}

In [3]:
def getFilesInDir(dir):
    """This function will return a list of files in the given dir"""
    files = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(dir):
        for file in f:
            files.append(os.path.join(r, file))
    return files

def validateImagePairs(imPairTuple, width):
    """This function reads the image and resizes it to the closest %32=0 for the specified width"""
    imgDir , annotDir = imPairTuple
    img = imio.imread(imgDir)
    annot = imio.imread(annotDir)
    
    
    widthScaleRatio = width / img.shape[1]
    heightNew = (widthScaleRatio * (img.shape[0]))
    heightNew = heightNew - (heightNew%32)
    
    annotNew = resize(annot, (heightNew, width),mode='edge', anti_aliasing=False,
                               anti_aliasing_sigma=None,preserve_range=True,
                               order=0).astype(int)
    
    return (resize(img, (heightNew, width), anti_aliasing=True),
            annotNew)

def augment_seg(imSegTuple, filterSeq):
    """This function applies a 'filter' to the input image and its annotation """
    img , seg = imSegTuple  
    img = img*255
    img = img.astype(np.uint8)
    aug_det = filterSeq.to_deterministic() 
    image_aug = aug_det.augment_image( img )
    segmap = ia.SegmentationMapOnImage( seg , nb_classes=len(np.unique(seg)), shape=img.shape )
    segmap_aug = aug_det.augment_segmentation_maps( segmap )
    segmap_aug = segmap_aug.get_arr_int()
    return (image_aug , segmap_aug)

def path_leaf(path):
    """This function gets the file name from a path"""
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

def saveAugmentOutputs(augmentImagePairs, originalFileNamePairs, suffex, outImagPath, outAnnotPath):
    """This function saves the augmented images"""
    for i in range(0, len(augmentImagePairs)):
        img = augmentImagePairs[i][0]
        fileName = path_leaf(originalFileNamePairs[i][0])
        fileNameSplit = fileName.split('.')
        imFileName = outImagPath+'\\'+fileNameSplit[0]+suffex+'.'+fileNameSplit[1]
        imio.imsave(imFileName, img)
        annot = augmentImagePairs[i][1]
        fileNameAnnot = path_leaf(originalFileNamePairs[i][1])
        fileNameAnnotSplit = fileNameAnnot.split('.')
        annotFileName = outAnnotPath+'\\'+fileNameAnnotSplit[0]+suffex+'.'+fileNameAnnotSplit[1]
        annot = annot.astype(np.uint8)
        imio.imwrite(annotFileName, annot)

def ShowAugmentCompare(originalTup, newTup):
    """This function shows a comparason of orginal vs augmented images and annotations"""
    fig, axes = plt.subplots(nrows=2, ncols=2)
    ax = axes.ravel()
    ax[0].imshow(originalTup[0])
    ax[0].set_title("Original image")
    ax[1].imshow(newTup[0])
    ax[1].set_title("New image")
    ax[2].imshow(originalTup[1], cmap='gray')
    ax[2].set_title("Original annotation")
    ax[3].imshow(newTup[1], cmap='gray')
    ax[3].set_title("New annotation")
    plt.tight_layout()
    plt.show()
    

def getSegClasses(seg):
    segUnique = np.unique(seg)
    #print(segUnique)
    output = {}
    for i in range(0,len(segUnique)):
        output[segUnique[i]] = i
    return output
        
    
def flattenSegImage(seg, classes):
    k = np.array(list(classes.keys()))
    v = np.array(list(classes.values()))
    sidx = k.argsort()
    output = v[sidx[np.searchsorted(k,seg,sorter=sidx)]]
    
    #print(stats.describe(seg).mean)
    #print(stats.describe(output).mean)
    return output

def processImages(filterCol, inputImagePairs, filePairPaths, dataPathImg, dataPathAnnot):    
    for key, value in tqdm(filterCol.items()):
        if(value==None):
            #print(key)
            saveAugmentOutputs(inputImagePairs, filePairPaths, key, dataPathImg, dataPathAnnot)
        else:
            #print('Applying filter \''+ key +'\' to input')
            outputAugment = []
            for tup in inputImagePairs:
                outputAugment.append(augment_seg(tup, value))
            #print('Saving output of filter \''+ key +'\'')
            saveAugmentOutputs(outputAugment, filePairPaths, key, dataPathImg, dataPathAnnot)


In [47]:
# This will create a list of tuples for the input images, it will associate an image with 
# its annotation given the naming convention is followed

print('Reading and resizing input files...')

WIDTH = 416

filePairPathsTrain = list(zip(getFilesInDir(dataPaths['InputImageTrain']), getFilesInDir(dataPaths['InputAnnotTrain'])))
inputImagePairsTrain = []
for tup in filePairPathsTrain:
    inputImagePairsTrain.append(validateImagePairs(tup,WIDTH))
    
print('Train: Read '+str(len(inputImagePairsTrain)*2)+' files.')

filePairPathsTest = list(zip(getFilesInDir(dataPaths['InputImageTest']), getFilesInDir(dataPaths['InputAnnotTest'])))
inputImagePairsTest = []
for tup in filePairPathsTest:
    inputImagePairsTest.append(validateImagePairs(tup,WIDTH))
    
print('Test: Read '+str(len(inputImagePairsTest)*2)+' files.')

Reading and resizing input files...
Train: Read 34 files.
Test: Read 16 files.


In [31]:
for i in range(0, len(inputImagePairsTrain)):
    print(getFilesInDir(dataPaths['InputAnnotTrain'])[i])
    print(np.unique((inputImagePairsTrain[i][1])))

C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Data\Train\Annotations\1001.png
[ 0  1  2  7 18 19 22 27 31 37 41]
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Data\Train\Annotations\1006.png
[ 0  1  2  7 18 19 22 27 31 37 41]
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Data\Train\Annotations\2001.png
[ 0  1  2  8 19 22 28 31 36 37]
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Data\Train\Annotations\2003.png
[ 0  1  2  8 19 22 28 31 36 37]
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Data\Train\Annotations\3001.png
[ 0  1  2  9 10 11 12 13 14 29 30 33 39 40 42]
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Data\Train\Annotations\4001.png
[ 0  1  2 14 15 19 22 38 42]
C:\Users\theza\D

In [48]:
# See https://github.com/aleju/imgaug#example-images for details

print('Createing filters...')

### Expriment 1, 2, 3
filters1 = {
    'a':None
}

### Expriment 4, 5, 6
filters2 = {
    'a':None,
    'b':iaa.Sequential([iaa.MotionBlur(k=5,angle=[-90, 90]),]), # horizontally flip ,
    'c':iaa.Sequential([iaa.Fliplr(1),]), # horizontally flip ,
    'd':iaa.Sequential([iaa.Flipud(1),]), # vertically flip ,
    'e':iaa.Sequential([iaa.PerspectiveTransform(scale=0.01, keep_size=True),]),# Perspective Transform,
    'f':iaa.Sequential([iaa.PiecewiseAffine(scale=0.01),]),# Piecewise Affine
    'g':iaa.Sequential([iaa.Multiply((0.9, 1.1)),]),# Make image brighter/darker
    'h':iaa.Sequential([iaa.AddToHueAndSaturation((-10, 10)),iaa.PiecewiseAffine(scale=0.01), iaa.PerspectiveTransform(scale=0.01, keep_size=True)]),# Shift the hue,
    'i':iaa.Sequential([iaa.MotionBlur(k=5,angle=[-90, 90]),iaa.Fliplr(1),iaa.AddToHueAndSaturation((-10, 10))]),
    'j':iaa.Sequential([iaa.MotionBlur(k=5,angle=[-90, 90]),iaa.Fliplr(1),iaa.Flipud(1)]),
    'k':iaa.Sequential([iaa.MotionBlur(k=5,angle=[-90, 90]),iaa.Fliplr(1),iaa.Flipud(1),iaa.PerspectiveTransform(scale=0.01, keep_size=True)]),
    'l':iaa.Sequential([iaa.Fliplr(1),iaa.Flipud(1),iaa.PerspectiveTransform(scale=0.01, keep_size=True),iaa.PiecewiseAffine(scale=0.01)]),
    'm':iaa.Sequential([iaa.Flipud(1),iaa.PerspectiveTransform(scale=0.01, keep_size=True),iaa.PiecewiseAffine(scale=0.01),iaa.GammaContrast((0.9, 1.1))]),
    'n':iaa.Sequential([iaa.PerspectiveTransform(scale=0.01, keep_size=True),iaa.PiecewiseAffine(scale=0.01),iaa.GammaContrast((0.9, 1.1)),iaa.AddToHueAndSaturation((-10, 10))]),
    'o':iaa.Sequential([iaa.PiecewiseAffine(scale=0.01),iaa.AddToHueAndSaturation((-10, 10))]),
    'p':iaa.Sequential([iaa.Flipud(1),iaa.PerspectiveTransform(scale=0.01, keep_size=True),iaa.PiecewiseAffine(scale=0.01),iaa.AddToHueAndSaturation((-10, 10))]),
    'q':iaa.Sequential([iaa.Fliplr(1),iaa.AddToHueAndSaturation((-10, 10))]),
    'r':iaa.Sequential([iaa.MotionBlur(k=5,angle=[-90, 90]),iaa.PerspectiveTransform(scale=0.01, keep_size=True),iaa.GammaContrast((0.9, 1.1)),iaa.AddToHueAndSaturation((-10, 10))]),
    's':iaa.Sequential([iaa.Fliplr(1),iaa.Flipud(1), iaa.GammaContrast((0.9, 1.1)),iaa.AddToHueAndSaturation((-10, 10))]),
    't':iaa.Sequential([iaa.Fliplr(1),iaa.Flipud(1), iaa.PiecewiseAffine(scale=0.01),iaa.AddToHueAndSaturation((-10, 10))]),
    'u':iaa.Sequential([iaa.Fliplr(1),iaa.Flipud(1), iaa.PerspectiveTransform(scale=0.01, keep_size=True),iaa.Multiply((0.9, 1.1))]),
    'v':iaa.Sequential([iaa.MotionBlur(k=5,angle=[-90, 90]),iaa.AddToHueAndSaturation((-10, 10))]),
    'w':iaa.Sequential([iaa.Flipud(1),iaa.PerspectiveTransform(scale=0.01, keep_size=True)]),
    'x':iaa.Sequential([iaa.Fliplr(1),iaa.AddToHueAndSaturation((-10, 10))]),
    'y':iaa.Sequential([iaa.PiecewiseAffine(scale=0.01),iaa.GammaContrast((0.9, 1.1))]),
    'z':iaa.Sequential([iaa.MotionBlur(k=5,angle=[-90, 90]),iaa.PerspectiveTransform(scale=0.01, keep_size=True),iaa.GammaContrast((0.9, 1.1)),iaa.AddToHueAndSaturation((-10, 10))])
}

### Expriment 8,
filters3 = {
    'a':None,
    'b':iaa.Sequential([iaa.Affine(rotate=(0, 20), mode='wrap')]),
    'c':iaa.Sequential([iaa.Affine(rotate=(40, 60), mode='wrap')]),
    'd':iaa.Sequential([iaa.Affine(rotate=(80, 100), mode='wrap')]),
    'e':iaa.Sequential([iaa.Affine(rotate=(120, 140), mode='wrap')]),
    'f':iaa.Sequential([iaa.Affine(rotate=(160, 180), mode='wrap')]),
    'g':iaa.Sequential([iaa.Affine(rotate=(200, 220), mode='wrap')]),
    'h':iaa.Sequential([iaa.Affine(rotate=(240, 260), mode='wrap')]),
    'i':iaa.Sequential([iaa.Affine(rotate=(280, 300), mode='wrap')]),
    'j':iaa.Sequential([iaa.Affine(rotate=(320, 340), mode='wrap')]),
    'k':iaa.Sequential([iaa.Affine(rotate=(20, 40), mode='wrap'),iaa.Fliplr(1)]),
    'l':iaa.Sequential([iaa.Affine(rotate=(60, 80), mode='wrap'),iaa.Fliplr(1)]),
    'm':iaa.Sequential([iaa.Affine(rotate=(100, 120), mode='wrap'),iaa.Fliplr(1)]),
    'n':iaa.Sequential([iaa.Affine(rotate=(140, 160), mode='wrap'),iaa.Fliplr(1)]),
    'm':iaa.Sequential([iaa.Affine(rotate=(180, 200), mode='wrap'),iaa.Fliplr(1)]),
    'o':iaa.Sequential([iaa.Affine(rotate=(220, 240), mode='wrap'),iaa.Fliplr(1)]),
    'p':iaa.Sequential([iaa.Affine(rotate=(260, 280), mode='wrap'),iaa.Fliplr(1)]),
    'q':iaa.Sequential([iaa.Affine(rotate=(300, 320), mode='wrap'),iaa.Fliplr(1)]),
    'r':iaa.Sequential([iaa.Affine(rotate=(340, 360), mode='wrap'),iaa.Fliplr(1)])
}

### Expriment 8,
filters4 = {
    'a01':None,
    'a02':iaa.Sequential([iaa.Affine(rotate=(0, 10), mode='wrap')]),
    'a03':iaa.Sequential([iaa.Affine(rotate=(10, 20), mode='wrap'),iaa.Fliplr(1)]),
    'a04':iaa.Sequential([iaa.Affine(rotate=(20, 30), mode='wrap')]),
    'a05':iaa.Sequential([iaa.Affine(rotate=(30, 40), mode='wrap'),iaa.Fliplr(1)]),
    'a06':iaa.Sequential([iaa.Affine(rotate=(40, 50), mode='wrap')]),
    'a07':iaa.Sequential([iaa.Affine(rotate=(50, 60), mode='wrap'),iaa.Fliplr(1)]),
    'a08':iaa.Sequential([iaa.Affine(rotate=(60, 70), mode='wrap')]),
    'a09':iaa.Sequential([iaa.Affine(rotate=(70, 80), mode='wrap'),iaa.Fliplr(1)]),
    'a10':iaa.Sequential([iaa.Affine(rotate=(80, 90), mode='wrap')]),
    'a11':iaa.Sequential([iaa.Affine(rotate=(90, 100), mode='wrap'),iaa.Fliplr(1)]),
    'a12':iaa.Sequential([iaa.Affine(rotate=(100, 110), mode='wrap')]),
    'a13':iaa.Sequential([iaa.Affine(rotate=(110, 120), mode='wrap'),iaa.Fliplr(1)]),
    'a14':iaa.Sequential([iaa.Affine(rotate=(120, 130), mode='wrap')]),
    'a15':iaa.Sequential([iaa.Affine(rotate=(130, 140), mode='wrap'),iaa.Fliplr(1)]),
    'a16':iaa.Sequential([iaa.Affine(rotate=(140, 150), mode='wrap')]),
    'a17':iaa.Sequential([iaa.Affine(rotate=(150, 160), mode='wrap'),iaa.Fliplr(1)]),
    'a18':iaa.Sequential([iaa.Affine(rotate=(160, 170), mode='wrap')]),
    'a19':iaa.Sequential([iaa.Affine(rotate=(170, 180), mode='wrap'),iaa.Fliplr(1)]),
    'a20':iaa.Sequential([iaa.Affine(rotate=(180, 190), mode='wrap')]),
    'a21':iaa.Sequential([iaa.Affine(rotate=(190, 200), mode='wrap'),iaa.Fliplr(1)]),
    'a22':iaa.Sequential([iaa.Affine(rotate=(200, 210), mode='wrap')]),
    'a23':iaa.Sequential([iaa.Affine(rotate=(210, 220), mode='wrap'),iaa.Fliplr(1)]),
    'a24':iaa.Sequential([iaa.Affine(rotate=(220, 230), mode='wrap')]),
    'a25':iaa.Sequential([iaa.Affine(rotate=(230, 240), mode='wrap'),iaa.Fliplr(1)]),
    'a26':iaa.Sequential([iaa.Affine(rotate=(240, 250), mode='wrap')]),
    'a27':iaa.Sequential([iaa.Affine(rotate=(250, 260), mode='wrap'),iaa.Fliplr(1)]),
    'a28':iaa.Sequential([iaa.Affine(rotate=(260, 270), mode='wrap')]),
    'a29':iaa.Sequential([iaa.Affine(rotate=(270, 280), mode='wrap'),iaa.Fliplr(1)]),
    'a30':iaa.Sequential([iaa.Affine(rotate=(280, 290), mode='wrap')]),
    'a31':iaa.Sequential([iaa.Affine(rotate=(290, 300), mode='wrap'),iaa.Fliplr(1)]),
    'a32':iaa.Sequential([iaa.Affine(rotate=(300, 310), mode='wrap')]),
    'a33':iaa.Sequential([iaa.Affine(rotate=(310, 320), mode='wrap'),iaa.Fliplr(1)]),
    'a34':iaa.Sequential([iaa.Affine(rotate=(320, 330), mode='wrap')]),
    'a35':iaa.Sequential([iaa.Affine(rotate=(330, 340), mode='wrap'),iaa.Fliplr(1)]),
    'a36':iaa.Sequential([iaa.Affine(rotate=(340, 350), mode='wrap')]),
    'a37':iaa.Sequential([iaa.Affine(rotate=(350, 360), mode='wrap'),iaa.Fliplr(1)])
}

### Expriment 8,
filters5 = {
    'a01':None,
    'a02':iaa.Sequential([iaa.Affine(rotate=(0, 5), mode='wrap')]),
    'a03':iaa.Sequential([iaa.Affine(rotate=(5, 10), mode='wrap'),iaa.Fliplr(1)]),
    'a04':iaa.Sequential([iaa.Affine(rotate=(10, 15), mode='wrap'),iaa.PerspectiveTransform(scale=0.01, keep_size=True)]),
    'a05':iaa.Sequential([iaa.Affine(rotate=(15, 20), mode='wrap',translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}),iaa.Fliplr(1)]),
    'a06':iaa.Sequential([iaa.Affine(rotate=(20, 25), mode='wrap')]),
    'a07':iaa.Sequential([iaa.Affine(rotate=(25, 30), mode='wrap'),iaa.Fliplr(1)]),
    'a08':iaa.Sequential([iaa.Affine(rotate=(30, 35), mode='wrap')]),
    'a09':iaa.Sequential([iaa.Affine(rotate=(35, 40), mode='wrap'),iaa.PerspectiveTransform(scale=0.01, keep_size=True),iaa.Fliplr(1)]),
    'a10':iaa.Sequential([iaa.Affine(rotate=(40, 45), mode='wrap',translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)})]),
    'a11':iaa.Sequential([iaa.Affine(rotate=(45, 50), mode='wrap'),iaa.Fliplr(1)]),
    'a12':iaa.Sequential([iaa.Affine(rotate=(50, 55), mode='wrap')]),
    'a13':iaa.Sequential([iaa.Affine(rotate=(55, 60), mode='wrap'),iaa.Fliplr(1)]),
    'a14':iaa.Sequential([iaa.Affine(rotate=(60, 65), mode='wrap'),iaa.PerspectiveTransform(scale=0.01, keep_size=True)]),
    'a15':iaa.Sequential([iaa.Affine(rotate=(65, 70), mode='wrap',translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}),iaa.Fliplr(1)]),
    'a16':iaa.Sequential([iaa.Affine(rotate=(70, 75), mode='wrap')]),
    'a17':iaa.Sequential([iaa.Affine(rotate=(75, 80), mode='wrap'),iaa.Fliplr(1)]),
    'a18':iaa.Sequential([iaa.Affine(rotate=(80, 85), mode='wrap')]),
    'a19':iaa.Sequential([iaa.Affine(rotate=(85, 90), mode='wrap'),iaa.Fliplr(1),iaa.PerspectiveTransform(scale=0.01, keep_size=True)]),
    'a20':iaa.Sequential([iaa.Affine(rotate=(90, 95), mode='wrap',translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)})]),
    'a21':iaa.Sequential([iaa.Affine(rotate=(95, 100), mode='wrap'),iaa.Fliplr(1)]),
    'a22':iaa.Sequential([iaa.Affine(rotate=(100, 105), mode='wrap')]),
    'a23':iaa.Sequential([iaa.Affine(rotate=(105, 110), mode='wrap'),iaa.Fliplr(1)]),
    'a24':iaa.Sequential([iaa.Affine(rotate=(110, 115), mode='wrap'),iaa.PerspectiveTransform(scale=0.01, keep_size=True)]),
    'a25':iaa.Sequential([iaa.Affine(rotate=(115, 120), mode='wrap',translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}),iaa.Fliplr(1)]),
    'a26':iaa.Sequential([iaa.Affine(rotate=(120, 125), mode='wrap')]),
    'a27':iaa.Sequential([iaa.Affine(rotate=(125, 130), mode='wrap'),iaa.Fliplr(1)]),
    'a28':iaa.Sequential([iaa.Affine(rotate=(130, 135), mode='wrap')]),
    'a29':iaa.Sequential([iaa.Affine(rotate=(135, 140), mode='wrap'),iaa.Fliplr(1),iaa.PerspectiveTransform(scale=0.01, keep_size=True)]),
    'a30':iaa.Sequential([iaa.Affine(rotate=(140, 145), mode='wrap',translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)})]),
    'a31':iaa.Sequential([iaa.Affine(rotate=(145, 150), mode='wrap'),iaa.Fliplr(1)]),
    'a32':iaa.Sequential([iaa.Affine(rotate=(150, 155), mode='wrap')]),
    'a33':iaa.Sequential([iaa.Affine(rotate=(155, 160), mode='wrap'),iaa.Fliplr(1)]),
    'a34':iaa.Sequential([iaa.Affine(rotate=(160, 165), mode='wrap'),iaa.PerspectiveTransform(scale=0.01, keep_size=True)]),
    'a35':iaa.Sequential([iaa.Affine(rotate=(165, 170), mode='wrap',translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}),iaa.Fliplr(1)]),
    'a36':iaa.Sequential([iaa.Affine(rotate=(170, 175), mode='wrap')]),
    'a37':iaa.Sequential([iaa.Affine(rotate=(175, 180), mode='wrap'),iaa.Fliplr(1)]),
    'a38':iaa.Sequential([iaa.Affine(rotate=(180, 185), mode='wrap')]),
    'a39':iaa.Sequential([iaa.Affine(rotate=(185, 190), mode='wrap'),iaa.Fliplr(1),iaa.PerspectiveTransform(scale=0.01, keep_size=True)]),
    'a40':iaa.Sequential([iaa.Affine(rotate=(190, 195), mode='wrap',translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)})]),
    'a41':iaa.Sequential([iaa.Affine(rotate=(195, 200), mode='wrap'),iaa.Fliplr(1)]),
    'a42':iaa.Sequential([iaa.Affine(rotate=(200, 205), mode='wrap')]),
    'a43':iaa.Sequential([iaa.Affine(rotate=(205, 210), mode='wrap'),iaa.Fliplr(1)]),
    'a44':iaa.Sequential([iaa.Affine(rotate=(210, 215), mode='wrap'),iaa.PerspectiveTransform(scale=0.01, keep_size=True)]),
    'a45':iaa.Sequential([iaa.Affine(rotate=(215, 220), mode='wrap',translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}),iaa.Fliplr(1)]),
    'a46':iaa.Sequential([iaa.Affine(rotate=(220, 225), mode='wrap')]),
    'a47':iaa.Sequential([iaa.Affine(rotate=(225, 230), mode='wrap'),iaa.Fliplr(1)]),
    'a48':iaa.Sequential([iaa.Affine(rotate=(230, 235), mode='wrap')]),
    'a49':iaa.Sequential([iaa.Affine(rotate=(235, 240), mode='wrap'),iaa.Fliplr(1),iaa.PerspectiveTransform(scale=0.01, keep_size=True)]),
    'a50':iaa.Sequential([iaa.Affine(rotate=(240, 245), mode='wrap',translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)})]),
    'a51':iaa.Sequential([iaa.Affine(rotate=(245, 250), mode='wrap'),iaa.Fliplr(1)]),
    'a52':iaa.Sequential([iaa.Affine(rotate=(250, 255), mode='wrap')]),
    'a53':iaa.Sequential([iaa.Affine(rotate=(255, 260), mode='wrap'),iaa.Fliplr(1)]),
    'a54':iaa.Sequential([iaa.Affine(rotate=(260, 265), mode='wrap'),iaa.PerspectiveTransform(scale=0.01, keep_size=True)]),
    'a55':iaa.Sequential([iaa.Affine(rotate=(265, 270), mode='wrap',translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}),iaa.Fliplr(1)]),
    'a56':iaa.Sequential([iaa.Affine(rotate=(270, 275), mode='wrap')]),
    'a57':iaa.Sequential([iaa.Affine(rotate=(275, 280), mode='wrap'),iaa.Fliplr(1)]),
    'a58':iaa.Sequential([iaa.Affine(rotate=(280, 285), mode='wrap')]),
    'a59':iaa.Sequential([iaa.Affine(rotate=(285, 290), mode='wrap'),iaa.Fliplr(1),iaa.PerspectiveTransform(scale=0.01, keep_size=True)]),
    'a60':iaa.Sequential([iaa.Affine(rotate=(290, 295), mode='wrap',translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)})]),
    'a61':iaa.Sequential([iaa.Affine(rotate=(295, 300), mode='wrap'),iaa.Fliplr(1)]),
    'a62':iaa.Sequential([iaa.Affine(rotate=(300, 305), mode='wrap')]),
    'a63':iaa.Sequential([iaa.Affine(rotate=(305, 310), mode='wrap'),iaa.Fliplr(1)]),
    'a64':iaa.Sequential([iaa.Affine(rotate=(310, 315), mode='wrap'),iaa.PerspectiveTransform(scale=0.01, keep_size=True)]),
    'a65':iaa.Sequential([iaa.Affine(rotate=(315, 320), mode='wrap',translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}),iaa.Fliplr(1)]),
    'a66':iaa.Sequential([iaa.Affine(rotate=(320, 325), mode='wrap')]),
    'a67':iaa.Sequential([iaa.Affine(rotate=(325, 330), mode='wrap'),iaa.Fliplr(1)]),
    'a68':iaa.Sequential([iaa.Affine(rotate=(330, 335), mode='wrap')]),
    'a69':iaa.Sequential([iaa.Affine(rotate=(335, 340), mode='wrap'),iaa.Fliplr(1),iaa.PerspectiveTransform(scale=0.01, keep_size=True)]),
    'a70':iaa.Sequential([iaa.Affine(rotate=(340, 345), mode='wrap',translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)})]),
    'a71':iaa.Sequential([iaa.Affine(rotate=(345, 350), mode='wrap'),iaa.Fliplr(1)]),
    'a72':iaa.Sequential([iaa.Affine(rotate=(350, 355), mode='wrap')]),
    'a73':iaa.Sequential([iaa.Affine(rotate=(355, 360), mode='wrap'),iaa.Fliplr(1)])
}


Createing filters...


In [49]:
print('Processing train images')
processImages(filters5, inputImagePairsTrain, filePairPathsTrain, dataPaths['OutputImageTrain'],dataPaths['OutputAnnotTrain'])

Processing train images


  0%|                                                                                           | 0/73 [00:00<?, ?it/s]Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

In [50]:
print('Processing test images')
processImages(filters1, inputImagePairsTest, filePairPathsTest, dataPaths['OutputImageTest'],dataPaths['OutputAnnotTest'])

Processing test images


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

In [51]:
from sklearn.model_selection import train_test_split # setting up the test and train data

print('Spilliting train and test sets')
fileOutputPathsTrain = list(zip(getFilesInDir(dataPaths['OutputImageTrain']),
                                getFilesInDir(dataPaths['OutputAnnotTrain'])))
dfTrain = pd.DataFrame(fileOutputPathsTrain, columns=['X', 'y'])
X_train = dfTrain['X']
y_train = dfTrain['y']

fileOutputPathsTest = list(zip(getFilesInDir(dataPaths['OutputImageTest']),
                                getFilesInDir(dataPaths['OutputAnnotTest'])))
dfTest = pd.DataFrame(fileOutputPathsTest, columns=['X', 'y'])
X_test = dfTest['X']
y_test = dfTest['y']

Spilliting train and test sets


In [52]:
from shutil import copyfile
import errno

def copyFileToDir(df, path):
    for file in df.values:
        print(file)
        dest = path+'\\'
        print(dest)
        dest = dest+path_leaf(file)
        os.makedirs(os.path.dirname(dest), exist_ok=True)
        copyfile(file, dest)

In [53]:
print('Copying train/test output')
copyFileToDir(X_train,dataPaths['XTrain'])
copyFileToDir(X_test,dataPaths['XTest'])
copyFileToDir(y_train,dataPaths['yTrain'])
copyFileToDir(y_test,dataPaths['yTest'])

Copying train/test output
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Images\1001a01.jpg
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\XTrain\
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Images\1001a02.jpg
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\XTrain\
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Images\1001a03.jpg
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\XTrain\
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Images\1001a04.jpg
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\XT

C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Images\1001a34.jpg
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\XTrain\
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Images\1001a35.jpg
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\XTrain\
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Images\1001a36.jpg
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\XTrain\
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Images\1001a37.jpg
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\XTrain\
C:\Users\theza\Docum

C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Images\1001a67.jpg
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\XTrain\
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Images\1001a68.jpg
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\XTrain\
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Images\1001a69.jpg
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\XTrain\
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Images\1001a70.jpg
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\XTrain\
C:\Users\theza\Docum

C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Images\1006a27.jpg
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\XTrain\
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Images\1006a28.jpg
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\XTrain\
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Images\1006a29.jpg
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\XTrain\
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Images\1006a30.jpg
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\XTrain\
C:\Users\theza\Docum

C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Images\1006a60.jpg
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\XTrain\
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Images\1006a61.jpg
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\XTrain\
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Images\1006a62.jpg
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\XTrain\
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Images\1006a63.jpg
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\XTrain\
C:\Users\theza\Docum

...

C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Annotations\8005a16.png
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\yTrain\
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Annotations\8005a17.png
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\yTrain\
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Annotations\8005a18.png
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\yTrain\
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Annotations\8005a19.png
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\yTrain\


C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\yTrain\
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Annotations\8005a49.png
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\yTrain\
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Annotations\8005a50.png
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\yTrain\
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Annotations\8005a51.png
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\yTrain\
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Annotations\8005a52.png


C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Annotations\8007a08.png
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\yTrain\
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Annotations\8007a09.png
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\yTrain\
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Annotations\8007a10.png
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\yTrain\
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Annotations\8007a11.png
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\yTrain\


C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\yTrain\
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Annotations\8007a41.png
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\yTrain\
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Annotations\8007a42.png
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\yTrain\
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Annotations\8007a43.png
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\yTrain\
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Annotations\8007a44.png


C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Train\Annotations\8007a73.png
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\yTrain\
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Test\Annotations\1005a.png
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\yTest\
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Test\Annotations\2002a.png
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\yTest\
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\Output\Test\Annotations\3004a.png
C:\Users\theza\Documents\Uni\MIT\2019\TP\Project\Meal-Compliance-Project\Additional-Data-Creator\OutputData\yTest\
C:\Users\the